In [2]:
import graphistry
import pandas as pd
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", personal_key_id="YIB753L14J", personal_key_secret="LDQ0JZUKMJYTVWNY")


In [3]:
df = pd.read_csv('./network_files/network1.csv',sep="\t")

In [4]:
df["source_ner"] = df.rel.str.split("-",expand=True)[0]

In [5]:
df["target_ner"] = df.rel.str.split("-",expand=True)[1].str.split(":",expand=True)[0]

In [6]:
df

,source,target,rel,re,source_ner,target_ner
0,csur p902,feces of a patient,STRAIN-ISOLATE:INHABITS,INHABITS,STRAIN,ISOLATE
1,csur p902,intensive care,STRAIN-ISOLATE:INHABITS,INHABITS,STRAIN,ISOLATE
2,prevotella intermedia 17,human periodontal pocket,STRAIN-ISOLATE:INHABITS,INHABITS,STRAIN,ISOLATE
3,prevotella intermedia atcc 25611,human periodontal pocket,STRAIN-ISOLATE:INHABITS,INHABITS,STRAIN,ISOLATE
4,nccp 16833,clinical specimen,STRAIN-ISOLATE:INHABITS,INHABITS,STRAIN,ISOLATE
...,...,...,...,...,...,...
36821,shewanella oneidensis mr 1,palladium (,STRAIN-COMPOUND:PRODUCES,PRODUCES,STRAIN,COMPOUND
36822,methylosinus trichosporium ob3b,copper,STRAIN-COMPOUND:PRODUCES,PRODUCES,STRAIN,COMPOUND
36823,bifidobacterium animalis subsp lactis bb 12,te,STRAIN-COMPOUND:PRODUCES,PRODUCES,STRAIN,COMPOUND
36824,bifidobacterium animalis subsp lactis bb 12,n,STRAIN-COMPOUND:PRODUCES,PRODUCES,STRAIN,COMPOUND


In [7]:
g = graphistry.edges(df, 'source', 'target',directed=True)

In [8]:
nodes_df = pd.concat([
	df[["source","source_ner"]].rename(columns={"source":"node_id","source_ner":"type"}),
	df[['target','target_ner']].rename(columns={"target":"node_id","target_ner":"type"})
	])

In [12]:
import matplotlib.cm as cm

colors = cm.get_cmap('Set3').colors
print(colors)


((0.5529411764705883, 0.8274509803921568, 0.7803921568627451), (1.0, 1.0, 0.7019607843137254), (0.7450980392156863, 0.7294117647058823, 0.8549019607843137), (0.984313725490196, 0.5019607843137255, 0.4470588235294118), (0.5019607843137255, 0.6941176470588235, 0.8274509803921568), (0.9921568627450981, 0.7058823529411765, 0.3843137254901961), (0.7019607843137254, 0.8705882352941177, 0.4117647058823529), (0.9882352941176471, 0.803921568627451, 0.8980392156862745), (0.8509803921568627, 0.8509803921568627, 0.8509803921568627), (0.7372549019607844, 0.5019607843137255, 0.7411764705882353), (0.8, 0.9215686274509803, 0.7725490196078432), (1.0, 0.9294117647058824, 0.43529411764705883))


/tmp/ipykernel_202051/989688878.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = cm.get_cmap('Set3').colors


In [13]:
color_codes = [matplotlib.colors.rgb2hex(color) for color in colors]
print(color_codes)


['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']


In [15]:
g2 = (g
      .nodes(nodes_df, 'node_id')

      # 'red', '#f00', '#ff0000'
      .encode_point_color('type', categorical_mapping={
          'STRAIN': 'blue',
          'ISOLATE': 'darkblue',
          'METABOLITE': 'yellow',
          'MEDIUM': 'orange',
          'DISEASE':"purple",
          'COMPOUND': "lightpurple",
          'PHENOTYPE': "red",
          'MORPHOLOGY': 'darkgreen',
          'EFFECT': 'green',
          'PROTEIN': 'lightblue',
            'GENE': 'lightgreen',
            'ORGANISM': 'lightyellow',
            'SPECIES': 'pink'
      }, default_mapping='red')

      # Gradient
      .encode_edge_color('re', palette=color_codes, as_continuous=False)


      .addStyle(bg={'color': '#eee'}, page={'title': 'My Graph'})

      # Options: https://hub.graphistry.com/docs/api/1/rest/url/
      .settings(url_params={'play': 1000, 'pointSize': 0.5})
)

g2.plot(as_files=True)